In [1]:
import sys
sys.path.append("../") 

import numpy as np
import scipy.linalg as slin
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

import LISA as l

import Wavelet as wv
import Glitch as gl
import Burst as bu

import Burst_MCMC as bumc
import MCMC_tools as mct


# constants
mHz = 1.0e-3
Hour = 3600.

In [2]:
Week = 3600.*24.*7.
dt   = 15.
Tobs = 2**np.ceil(np.log2(Week/dt))*dt
orb = l.Orbit(Tobs, dt=dt)
#t = np.arange(0.0, orb.Tobs, orb.dt) # set up the time of this orbit

A    = 2.0e-21
f0   = 3.0*mHz
tau  = 1.5*Hour
t0   = 0.5*orb.Tobs
phi0 = 1.0

theta = np.pi*3/4
phi   = np.pi*1.
psi   = np.pi*0.2
ellip = 0.

# Non-Dimensional parameters
paramsND = np.array([np.log(A), f0/mHz, t0/Week, tau/Week, phi0, \
                     np.cos(theta), phi, psi, ellip])

gw = bu.Burst(paramsND, orb)
gw.construct_detector_tensor()
gw.calculate_strain()
gw.TDI = gw.construct_TDI(orb)

In [ ]:
gw.calc_snr()
gw.adjust_snr(10)

modelX0 = bumc.Model(gw, orb) 
modelX0.get_loglkl(gw.TDI)
print(modelX0.loglkl)

PT = 1
flag = bumc.Flag(PT)

seed = 1
N_MCMC = 10**5
lkl, chain, props, max_model = bumc.Burst_MCMC(TDI_data=gw.TDI, Orbit=orb, modelX0=modelX0, \
                                                                seed=seed, N=N_MCMC, Flag=flag)

FF = (max_model.loglkl + 0.5*max_model.Burst.SNR**2)/max_model.Burst.SNR/modelX0.Burst.SNR
print("FF......... {}".format(FF))

  0%|          | 0/100000 [00:00<?, ?it/s]

50.00059392610618


 65%|██████▍   | 64797/100000 [01:58<01:04, 548.57it/s]

In [ ]:
n_bins = 30
mct.plot_chain(lkl[:,0], r'log L', n_bins)
mct.plot_chain(chain[0,:,0], r'$\log A$',   n_bins, np.log(gw.A))
mct.plot_chain(chain[1,:,0], r'$f_{0}$',    n_bins, f0/mHz)
mct.plot_chain(chain[2,:,0], r'$t_{0}$',    n_bins, t0/Week)
mct.plot_chain(chain[3,:,0], r'$\tau$',     n_bins, tau/Week)
mct.plot_chain(chain[4,:,0], r'$\phi_{0}$', n_bins, phi0)

mct.plot_chain(chain[5,:,0], r'$\cos \theta$',   n_bins, np.cos(theta))
mct.plot_chain(chain[6,:,0], r'$\phi$',          n_bins, phi)
mct.plot_chain(chain[7,:,0], r'$\psi$',          n_bins, psi)
mct.plot_chain(chain[8,:,0], r'$\epsilon$',      n_bins, ellip)


In [ ]:
for i in range(9):
    print(mct.get_autocorr_length(chain[i,:,0]))

In [ ]:
#Setup a grid
import scipy
x = np.arccos(chain[5,:,0])
y = chain[6,:,0]

plt.scatter(x,y)

plt.show()

In [ ]:
from scipy.stats import kde

fig, ax = plt.subplots(figsize=(8, 6))

# Create data
x, y = np.arccos(chain[5,:,0]), chain[6,:,0]
data = np.vstack((x,y)).T

nbins = 20

# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
k = kde.gaussian_kde(data.T)
xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi = k(np.vstack([xi.flatten(), yi.flatten()]))

# add shading
ax.set_title('2D Density with shading')
ax.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='gouraud', cmap=plt.cm.BuGn_r)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='mollweide')

x, y = np.arccos(chain[5,:,0]), chain[6,:,0]
xx = x - np.pi/2
yy = y -  np.pi

Lat, Lon = np.mgrid[xx.min():xx.max():nbins*1j, yy.min():yy.max():nbins*1j]
arr = k(np.vstack([Lon.flatten(), Lat.flatten()]))
arr = arr.reshape(Lat.shape)

im = ax.pcolormesh(Lon,Lat,arr, cmap=plt.cm.jet)

plt.show()

In [ ]:
plt.plot(lkl[:,0])
plt.plot(lkl[:,1])
plt.plot(lkl[:,2])
#plt.plot(lkl[:,3])
plt.show()